### Graham Kerr
#### graham.s.kerr@NASA.gov; kerrg@cua.edu

<H1 font size="+3" style="color:red">
Ly_HSR Project<br>
-- Plot He Pops
</H1>

<b>This notebook will load a RADYN simulation, and plot the He ionisation fractions, and the He population levels.</b> <br>
 > - This is to start investigating if the opacity increases in some simulations before the LyC emissivity increases, resulting in transient dimming of the continuum near 700-911A.
 > - Excited states of He II have continua heads near 911A, so increased opacity would absorb photons <911A. Is there an increase in the populations of those levels at the time we see dimming of the continuum? 



***Import Modules***

In [ ]:
##
## Import some modules
##

import sys
sys.path.insert(0,'/Users/gskerr1/Documents/Research/Python_Programs/radynpy/')
import radynpy 
from radynpy.utils import RadynMovie as RM

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
from matplotlib.ticker import LogLocator
from matplotlib import ticker
import matplotlib.colorbar as cb
import pandas as pd

import cmocean
import colorcet as ccet
import palettable as pal 
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

***Set up plot properties***

In [ ]:
##
## Plot properties (these are just personal preference)
##

#Avenir LT Std
font = {'family': 'Avenir LT Std',
        'color':  'black',
        'weight': 'medium',
        'size': 22,
        }
plot_params = {'ytick.direction': 'in', 
               'xtick.direction': 'in', 
               'xtick.minor.visible': True,
               'ytick.minor.visible': True,
               'xtick.major.size': 10, 'xtick.minor.size': 5,
               'ytick.major.size': 10, 'ytick.minor.size': 5,
               'ytick.right': True,
               'xtick.top': True,
               'ytick.major.width': 1.5,
               'xtick.major.width': 1.5,
               'ytick.minor.width': 1.5,
               'xtick.minor.width': 1.5,
               'axes.linewidth': 1.5,
               'axes.spines.top': True,
               'axes.spines.bottom': True,
               'axes.spines.left': True,
               'axes.spines.right': True,
               'axes.titlepad' : 18 }

plot_lg_params = {'legend.frameon': False}
#plt.rcParams.update({'font.size': font['size'], 'font.family':font['family'], 'font.weight':font['weight'], 'font.color':font['color']})

plt.rcParams.update({'font.size':font['size'], 'font.family':font['family'], 'font.weight':font['weight']})
plt.rcParams.update({'ytick.direction': plot_params['ytick.direction'],
                     'xtick.direction': plot_params['xtick.direction'],
                     'xtick.minor.visible': plot_params['xtick.minor.visible'],
                     'ytick.minor.visible': plot_params['ytick.minor.visible'],
                     'ytick.major.size':  plot_params['ytick.major.size'], 
                     'ytick.minor.size':  plot_params['ytick.minor.size'],
                     'xtick.major.size':  plot_params['xtick.major.size'],                                
                     'xtick.minor.size':  plot_params['xtick.minor.size'],
                     'ytick.right': plot_params['ytick.right'],
                     'xtick.top': plot_params['xtick.top'],
                     'ytick.major.width': plot_params['ytick.major.width'],
                     'xtick.major.width': plot_params['xtick.major.width'],
                     'ytick.minor.width': plot_params['ytick.minor.width'],
                     'xtick.minor.width': plot_params['xtick.minor.width'],                    
                     'axes.linewidth': plot_params['axes.linewidth'],
                     'axes.spines.top' : plot_params['axes.spines.top'],
                     'axes.spines.bottom' : plot_params['axes.spines.bottom'],
                     'axes.spines.left' : plot_params['axes.spines.left'],
                     'axes.spines.right' : plot_params['axes.spines.right'],
                     'axes.titlepad' : plot_params['axes.titlepad'],
                    })

plt.rcParams.update({'legend.frameon': plot_lg_params['legend.frameon']})

mpl.mathtext.SHRINK_FACTOR = 0.6
mpl.mathtext.GROW_FACTOR = 1 / 0.6





template = dict(
        layout = go.Layout(font = dict(family = "Rockwell", size = 16),
                           title_font = dict(family = "Rockwell", size = 20), 
                           plot_bgcolor = 'white',
                           paper_bgcolor = 'white',
                           xaxis = dict(
                                showexponent = 'all',
                                exponentformat = 'e',
                                tickangle = 0,
                                linewidth = 3,
                                showgrid = True,
                            ),
                            yaxis = dict(
                          showexponent = 'all',
                          exponentformat = 'e',
                                linewidth = 3,
                                showgrid = True,
                                anchor = 'free',
                                position = 0,
                                domain = [0.0,1]
                            ),
                            coloraxis_colorbar = dict(
                                thickness = 15,
                                tickformat = '0.2f',
                                ticks = 'outside',
                                titleside = 'right'
                            )
                            ))

***Load the RADYN Simulation(s) of interest***

In [ ]:
##
## Load the simulations
##
dir1 = '/Users/gskerr1/Documents/Research/FCHROMA_Grid/'
cdf1 = radynpy.cdf.LazyRadynData(dir1+'radyn_out.val3c_d8_1.0e11_t20s_15kev_fp')




***Extract the variables that we will need***

In [ ]:
## 
## Load the variables of interest
##

cdf1.load_var('z1')
cdf1.load_var('n1')
cdf1.load_var('tg1')
cdf1.load_var('time')
cdf1.load_var('irad')
cdf1.load_var('jrad')
cdf1.load_var('ielrad')
cdf1.load_var('label')
cdf1.load_var('ion')
cdf1.load_var('cont')
cdf1.load_var('alamb')

***What are the levels we want to plot?***<br>
We search below for which transitions are free-bound from He II -> He III.<br>
> cont -- is '1' for a free-bound transition, '0' for a bound-bound transition<br>
> irad -- the lower level of the transition. It runs from level '1' not '0', so need to add one to the level number.<br>

We then add those transitions to a list, and print the results

In [ ]:
iel = 3 #Helium
ion = 2 #He II
levs = (np.where(cdf1.ion[:,iel-1] == ion))[0] #indexing starts at zero so element #iel is indexed as iel-1
nlevs = len(levs) #number of levels 
rind = []
for i in range(nlevs):
    rind.append(np.where((cdf1.ielrad == iel) & (cdf1.irad == levs[i]+1) & (cdf1.cont == 1))[0][0])

## Print the wavelengths associated with the b-f transitions of He II    
print("Number of transitions = ",nlevs)
print("Level #", levs)
print("Wavelengths = ",cdf1.alamb[rind],'A')


***Makes some movies of the levels we are interested in***<br>

In [ ]:
t1 = 0
t2 = 200


dfout = RM.rmovie(cdf1.z1[t1:t2]/1e8, cdf1.n1[t1:t2,:,levs[0],iel-1], time = cdf1.time[t1:t2], 
                      xlog = False, ylog = True, 
                      xtitle = 'Height [Mm]', 
                      ytitle = 'Population [cm<sup>-3</sup>]', 
                      title = cdf1.label[levs[0],iel-1],
                      savefig= False,figname='radynfig.html')

In [ ]:
t1 = 0
t2 = 200
dfout = RM.rmovie(cdf1.z1[t1:t2]/1e8, cdf1.n1[t1:t2,:,levs[1],iel-1], time = cdf1.time[t1:t2], 
                      xlog = False, ylog = True, 
                      xtitle = 'Height [Mm]', 
                      ytitle = 'Population [cm<sup>-3</sup>]', 
                      title = cdf1.label[levs[1],iel-1],
                      savefig= False,figname='radynfig.html')

In [ ]:
t1 = 0
t2 = 200
dfout = RM.rmovie(cdf1.z1[t1:t2]/1e8, cdf1.n1[t1:t2,:,levs[2],iel-1], time = cdf1.time[t1:t2], 
                      xlog = False, ylog = True, 
                      xtitle = 'Height [Mm]', 
                      ytitle = 'Population [cm<sup>-3</sup>]', 
                      title = cdf1.label[levs[2],iel-1],
                      savefig= False,figname='radynfig.html')

***The following cells are basically what RadynMovie.py does behind the scenes but you can plot all three lines on the same figure*** <br>
> - This is pretty memory intensive though

In [ ]:
pop1 = 'He II gr'
pop2 = 'He II ex1'
pop3 = 'He II ex2'
height = 'Height [Mm]'
time = 'Time [s]'
timeind = 'Time index'
df_list = []
for i in range(len(cdf1.time)):
    data = {pop1:cdf1.n1[i,:,levs[0],iel-1],
        pop2:cdf1.n1[i,:,levs[1],iel-1],
        pop3:cdf1.n1[i,:,levs[2],iel-1],
        height: cdf1.z1[i,:]/1e8,
        time: cdf1.time[i],
        timeind: i
           }
    df_list.append(pd.DataFrame(data))
    
df = pd.concat(df_list)

In [ ]:
xtitle = 'Height [Mm]'
ytitle_gr = ['He II gr pop [cm<sup>-3</sup>]']

h1 = 700
w1 = 700

fig1 = px.line(df,
               x = df['Height [Mm]']/1e8, y = df.columns[:3],
#                animation_group = 'Time [s]',
               animation_frame = 'Time [s]',
               labels = dict(x = xtitle, y = ytitle_gr),
              log_x = False,
              log_y = True,
              template = template)

fig1.show()


In [ ]:
dfout = RM.rmovie(cdf1.z1/1e8, cdf1.tg1[:,:], time = cdf1.time, 
                      xlog = False, ylog = True, 
                      xtitle = 'Height [Mm]', 
                      ytitle = 'Temperature [K]', 
                      title = '3F10, d5, Ec15, FCHROMA',
                      savefig= True,figname='RadynMovie_ex_Tempr.html')